## Deploying CNN to AWS Sagemaker
reference https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/

In [ ]:
!pip uninstall tensorflow --yes
!pip install tensorflow==1.15.3

In [ ]:
import tensorflow as tf
print(tf.__version__)  

## Download file from S3

In [ ]:
import boto3
import os
import numpy as np
import pickle
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image, ImageOps
import sys
from io import BytesIO

s3 = boto3.resource('s3')
s3.Bucket('cnn-model-bucket').download_file('modelh5_epoch30_1', 'modelh5_epoch30_1')

## Load Model


In [ ]:
MODEL_LOCATION ='modelh5_epoch30_1'
from tensorflow.keras.models import load_model
model = load_model(MODEL_LOCATION)  

## Convert model into AWS format
Adapted from reference blog

In [ ]:

def convert_h5_to_aws(loaded_model):
    # import tensorflow as tf
    # if tf.executing_eagerly():
    #     tf.compat.v1.disable_eager_execution()
    from tensorflow.python.saved_model import builder
    from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
    from tensorflow.python.saved_model import tag_constants
    
    # This is the file structure which AWS expects. Cannot be changed. 
    model_version = '1'
    export_dir = 'export/Servo/' + model_version
    
    builder = builder.SavedModelBuilder(export_dir)
    
    signature = predict_signature_def(
        inputs={"inputs": loaded_model.input}, outputs={"gender": loaded_model.output[0], "age": loaded_model.output[1]})
    
    from tensorflow.keras import backend as K
    with K.get_session() as sess:
        builder.add_meta_graph_and_variables(
            sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
        builder.save()
    
    import tarfile
    with tarfile.open('model.tar.gz', mode='w:gz') as archive:
        archive.add('export', recursive=True)
        
convert_h5_to_aws(model)

## Upload to S3

In [ ]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

## Create SageMaker Model

In [ ]:
!touch train.py #create an empty python file

In [37]:
import boto3, re
from sagemaker import get_execution_role

role = get_execution_role()

from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  framework_version = '1.15',
                                  entry_point = 'train.py')

## Deploy to Sagemaker endpoint instance

In [ ]:
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                   instance_type='ml.m4.xlarge')
predictor.endpoint